In [1]:
!pip install -q kornia==0.6.12 pycolmap

In [101]:
# General utilities
import os
from tqdm import tqdm
from time import time
# from fastprogress import progress_bar
import gc
import math
import numpy as np
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
import matplotlib.pyplot as plt
import concurrent.futures

# CV/ML
import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
from PIL import Image
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

# 3D reconstruction
import pycolmap

print("Kornia version", K.__version__)
print("Pycolmap version", pycolmap.__version__)

Kornia version 0.6.12
Pycolmap version 0.6.1


# Global Configs

In [102]:
# Mode can only be train or test. This will be used to find the image directory.
# Use "test" for submission 
MODE = "train"
MODE = "test"

# Option to change path for local testing
is_local = True
is_local = False

if is_local:
    NUM_CORES = 2
    SRC = "image-matching-challenge-2024"
    MODEL_DIR = "kornia-local-feature-weights/"
    DISK_PATH = "loftr_disk.ckpt"
    HARDNET_PT = "kornia-local-feature-weights/hardnet8v2.pt"
else:
    NUM_CORES = 2
    SRC = "image-matching-challenge-2024"
    MODEL_DIR = "kornia-local-feature-weights/"
    DISK_PATH = "disk/pytorch/depth-supervision/1/loftr_outdoor.ckpt"
    HARDNET_PT = "hardnet8v2/hardnet8v2.pt"

LOG_MESSAGE = "Final submission"
MATCHES_CAP = None

DEBUG = True
DEBUG = False

# DEBUG_SCENE = ["cyprus", "kyiv-puppet-theater"]
# DEBUG_SCENE = ["cyprus"]
# DEBUG_SCENE = ["kyiv-puppet-theater"]
# DEBUG_SCENE = ["kyiv-puppet-theater", "cyprus", "wall", "chairs"]
DEBUG_SCENE = ["chairs"]
# DEBUG_SCENE = ["wall"]

# Longer edge limit of the input image
hardnet_res = 1600

MODEL_DICT = {
    "Keynet": {"enable":True, "resize_long_edge_to": hardnet_res, "pair_only": False},
    "GFTT": {"enable": True, "resize_long_edge_to": hardnet_res},
    "DoG": {"enable": True, "resize_long_edge_to": hardnet_res},
    "Harris": {"enable": True, "resize_long_edge_to": hardnet_res},
}

# Find fundamental matrix parameters
FM_PARAMS = {"ransacReprojThreshold": 10, "confidence": 0.9999, "maxIters": 50000, "removeOutliers": True}

# Remove a "match" if the number of matches is lower than MATCH_FILTER_RATIO*max_num_matches
# e.g. img1 and img2 have max 10000 matches with some other images, img2 and img1 only have 99 matches. The matches btw img1 and img2 won't be selected.
MATCH_FILTER_RATIO = 0.01

# for logging
LOG_DICT = dict()
LOG_DICT["mode"] = MODE
LOG_DICT["log_message"] = LOG_MESSAGE
LOG_DICT["matches_cap"] = MATCHES_CAP
LOG_DICT["debug"] = DEBUG
LOG_DICT["debug_scene"] = DEBUG_SCENE

if MODE == "test":
    DEBUG = False
device = torch.device("cuda")
print(torch.cuda.is_available())


True


# Get datadict from submission file

In [103]:
# Get datadict from csv.
if MODE == "train":
    sample_path = f"{SRC}/train/train_labels.csv"
else:
    sample_path = f"{SRC}/sample_submission.csv"

data_dict = {}
with open(sample_path, "r") as f:
    for i, l in enumerate(f):
        # Skip header.
        if l and i > 0:
            if MODE == "train":
                dataset, scene, image, _, _ = l.strip().split(",")
            else:
                image, dataset, scene, _, _ = l.strip().split(",")
            if dataset not in data_dict:
                data_dict[dataset] = {}
            if scene not in data_dict[dataset]:
                data_dict[dataset][scene] = []
            data_dict[dataset][scene].append(image)
            
all_scenes = []
scene_len = []
for dataset in data_dict:
    for scene in data_dict[dataset]:
        print(f"{dataset} / {scene} -> {len(data_dict[dataset][scene])} images")
        if DEBUG and (scene not in DEBUG_SCENE):
            continue
        all_scenes.append((dataset, scene))
        scene_len.append(len(data_dict[dataset][scene]))

# sort all scenes by length, lowest first
all_scenes = [x for _, x in sorted(zip(scene_len, all_scenes), reverse=True)]

# Print reconst order
print("Reconstruction order: ")
for scene in all_scenes:
    print(f" --{scene[0]} / {scene[1]}")

church / church -> 41 images
Reconstruction order: 
 --church / church


# Submission Utils

In [104]:
def arr_to_str(a):
    return ";".join([str(x) for x in a.reshape(-1)])


# Function to create a submission file.
def create_submission(out_results, data_dict, mode="test"):
    if mode == "train":
        file_name = "submission_train.csv"
    else:
        file_name = "submission.csv"

    with open(file_name, "w") as f:
        f.write("image_path,dataset,scene,rotation_matrix,translation_vector\n")
        for dataset in data_dict:
            if dataset in out_results:
                res = out_results[dataset]
            else:
                res = {}
            for scene in data_dict[dataset]:
                if scene in res:
                    scene_res = res[scene]
                else:
                    scene_res = {"R": {}, "t": {}}
                for image in data_dict[dataset][scene]:
                    if image in scene_res:
                        print(image)
                        R = scene_res[image]["R"].reshape(-1)
                        T = scene_res[image]["t"].reshape(-1)
                    else:
                        R = np.eye(3).reshape(-1)
                        T = np.zeros((3))
                    f.write(
                        f"{image},{dataset},{scene},{arr_to_str(R)},{arr_to_str(T)}\n"
                    )


# Image Loading and Resize

In [105]:
def load_torch_image(fname, device=torch.device("cpu")):
    # Load the image from the local file path
    img_np = cv2.imread(fname)
    img_rgb = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
    
    # Convert to grayscale
    gray_img = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
    
    # Perform bilateral filtering
    bilateral_filtered = cv2.bilateralFilter(gray_img, d=9, sigmaColor=75, sigmaSpace=75)
    
    # Find edges using Canny edge detection
    edges = cv2.Canny(bilateral_filtered, threshold1=0, threshold2=60)
    
    # Dilate the edges to enhance them
    dilated = cv2.dilate(edges, None)
    
    # Normalize and convert the dilated edges to 8-bit image
    dilated_norm = cv2.normalize(dilated, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
    
    # Blur the dilated image using a window of size 9x9
    blurred = cv2.GaussianBlur(dilated_norm, (9, 9), 0)
    
    # Convert the final processed image to tensor and normalize
    final_img = K.image_to_tensor(cv2.cvtColor(blurred, cv2.COLOR_GRAY2BGR), False).float() / 255.0
    final_img = K.color.bgr_to_rgb(final_img.to(device))
    
    # Convert original image to tensor and move to device
    original_img = K.image_to_tensor(img_rgb, False).float() / 255.0
    original_img = original_img.to(device)  

    return (0.6*final_img)+original_img


def resize_torch_image(
    timg, resize_long_edge_to=None, align=None, disable_enlarge=True
):
    h, w = timg.shape[2:]
    raw_size = torch.tensor(timg.shape[2:])
    if resize_long_edge_to is None:
        scale = 1
    else:
        scale = float(resize_long_edge_to) / float(max(raw_size[0], raw_size[1]))

    if disable_enlarge:
        scale = min(scale, 1)

    h_resized = int(h * scale)
    w_resized = int(w * scale)

    if align is not None:
        assert align > 0
        h_resized = h_resized - h_resized % align
        w_resized = w_resized - w_resized % align
    scale_h = h_resized / h
    scale_w = w_resized / w

    timg_resized = K.geometry.resize(timg, (h_resized, w_resized), antialias = True)
    return timg_resized, scale_h, scale_w


def get_roi_image(timg, roi):
    min_h = int(roi["roi_min_h"])
    min_w = int(roi["roi_min_w"])
    max_h = int(roi["roi_max_h"])
    max_w = int(roi["roi_max_w"])
    roi_img = timg[:, :, min_h:max_h, min_w:max_w]
    roi_w_scale = (max_w - min_w) / timg.shape[3]
    roi_h_scale = (max_h - min_h) / timg.shape[2]
    return roi_img, min_h, min_w

# Visualization Utils

In [106]:
# Visualzation block
def draw_keypoints(img, keypoints, color=(0, 255, 0)):
    max_edge = max(img.shape[0], img.shape[1])
    good_radius = 4
    for kp in keypoints:
        x, y = kp
        cv2.circle(img, (int(x), int(y)), color=color, radius=good_radius, thickness=-1)


def draw_roi(img, roi, color=(0, 255, 255)):
    x1, y1, x2, y2 = (
        roi["roi_min_w"],
        roi["roi_min_h"],
        roi["roi_max_w"],
        roi["roi_max_h"],
    )
    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color=color, thickness=2)


def plot_images_with_keypoints(fname1, fname2, kpts1, kpts2, matches, rois=None):
    print(fname1, fname2)
    # Draw keypoints on the images
    image1 = cv2.imread(fname1)
    image2 = cv2.imread(fname2)
    print(image1.shape, image2.shape)

    draw_keypoints(image1, kpts1)
    draw_keypoints(image2, kpts2)
    if rois is not None:
        draw_roi(image1, rois[0])
        draw_roi(image2, rois[1])
    print(image1.shape, image2.shape)
    print("Number of matches:", len(matches))
    print("Number of keypoints:", len(kpts1), len(kpts2))
    #print the first match
    print(matches[0])
    # Resize image1 and image2 to have the same smaller height
    display_h = 840
    h1, w1 = image1.shape[:2]
    h2, w2 = image2.shape[:2]
    # new_h = min(h1, h2)
    scale1 = display_h / h1
    scale2 = display_h / h2
    new_w1 = int(w1 * scale1)
    new_w2 = int(w2 * scale2)

    image1 = cv2.resize(image1, (new_w1, display_h))
    image2 = cv2.resize(image2, (new_w2, display_h))

    # Create a new image by horizontally concatenating the two images
    concatenated_img = cv2.hconcat([image1, image2])

    # Draw lines between the matching keypoints
    for match in matches:
        img1_idx = match[0]
        img2_idx = match[1]
        (x1, y1) = kpts1[img1_idx] * scale1
        (x2, y2) = kpts2[img2_idx] * scale2
        pt1 = (int(x1), int(y1))
        pt2 = (int(x2) + image1.shape[1], int(y2))
        cv2.line(concatenated_img, pt1, pt2, (0, 0, 255), 2)

    # Plot the concatenated image
    plt.figure(figsize=(20, 12))
    plt.imshow(cv2.cvtColor(concatenated_img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

# Colmap database

In [107]:
# Code to manipulate a colmap database.
# Forked from https://github.com/colmap/colmap/blob/dev/scripts/python/database.py

# Copyright (c) 2018, ETH Zurich and UNC Chapel Hill.
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
#     * Redistributions of source code must retain the above copyright
#       notice, this list of conditions and the following disclaimer.
#
#     * Redistributions in binary form must reproduce the above copyright
#       notice, this list of conditions and the following disclaimer in the
#       documentation and/or other materials provided with the distribution.
#
#     * Neither the name of ETH Zurich and UNC Chapel Hill nor the names of
#       its contributors may be used to endorse or promote products derived
#       from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
# ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDERS OR CONTRIBUTORS BE
# LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR
# CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF
# SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS
# INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN
# CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.
#
# Author: Johannes L. Schoenberger (jsch-at-demuc-dot-de)

# This script is based on an original implementation by True Price.

import sys
import sqlite3
import numpy as np


IS_PYTHON3 = sys.version_info[0] >= 3

MAX_IMAGE_ID = 2**31 - 1

CREATE_CAMERAS_TABLE = """CREATE TABLE IF NOT EXISTS cameras (
    camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    model INTEGER NOT NULL,
    width INTEGER NOT NULL,
    height INTEGER NOT NULL,
    params BLOB,
    prior_focal_length INTEGER NOT NULL)"""

CREATE_DESCRIPTORS_TABLE = """CREATE TABLE IF NOT EXISTS descriptors (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)"""

CREATE_IMAGES_TABLE = """CREATE TABLE IF NOT EXISTS images (
    image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL UNIQUE,
    camera_id INTEGER NOT NULL,
    prior_qw REAL,
    prior_qx REAL,
    prior_qy REAL,
    prior_qz REAL,
    prior_tx REAL,
    prior_ty REAL,
    prior_tz REAL,
    CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < {}),
    FOREIGN KEY(camera_id) REFERENCES cameras(camera_id))
""".format(
    MAX_IMAGE_ID
)

CREATE_TWO_VIEW_GEOMETRIES_TABLE = """
CREATE TABLE IF NOT EXISTS two_view_geometries (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    config INTEGER NOT NULL,
    F BLOB,
    E BLOB,
    H BLOB)
"""

CREATE_KEYPOINTS_TABLE = """CREATE TABLE IF NOT EXISTS keypoints (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)
"""

CREATE_MATCHES_TABLE = """CREATE TABLE IF NOT EXISTS matches (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB)"""

CREATE_NAME_INDEX = "CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)"

CREATE_ALL = "; ".join(
    [
        CREATE_CAMERAS_TABLE,
        CREATE_IMAGES_TABLE,
        CREATE_KEYPOINTS_TABLE,
        CREATE_DESCRIPTORS_TABLE,
        CREATE_MATCHES_TABLE,
        CREATE_TWO_VIEW_GEOMETRIES_TABLE,
        CREATE_NAME_INDEX,
    ]
)


def image_ids_to_pair_id(image_id1, image_id2):
    if image_id1 > image_id2:
        image_id1, image_id2 = image_id2, image_id1
    return image_id1 * MAX_IMAGE_ID + image_id2


def pair_id_to_image_ids(pair_id):
    image_id2 = pair_id % MAX_IMAGE_ID
    image_id1 = (pair_id - image_id2) / MAX_IMAGE_ID
    return image_id1, image_id2


def array_to_blob(array):
    if IS_PYTHON3:
        return array.tostring()
    else:
        return np.getbuffer(array)


def blob_to_array(blob, dtype, shape=(-1,)):
    if IS_PYTHON3:
        return np.fromstring(blob, dtype=dtype).reshape(*shape)
    else:
        return np.frombuffer(blob, dtype=dtype).reshape(*shape)


class COLMAPDatabase(sqlite3.Connection):
    @staticmethod
    def connect(database_path):
        return sqlite3.connect(database_path, factory=COLMAPDatabase)

    def __init__(self, *args, **kwargs):
        super(COLMAPDatabase, self).__init__(*args, **kwargs)

        self.create_tables = lambda: self.executescript(CREATE_ALL)
        self.create_cameras_table = lambda: self.executescript(CREATE_CAMERAS_TABLE)
        self.create_descriptors_table = lambda: self.executescript(
            CREATE_DESCRIPTORS_TABLE
        )
        self.create_images_table = lambda: self.executescript(CREATE_IMAGES_TABLE)
        self.create_two_view_geometries_table = lambda: self.executescript(
            CREATE_TWO_VIEW_GEOMETRIES_TABLE
        )
        self.create_keypoints_table = lambda: self.executescript(CREATE_KEYPOINTS_TABLE)
        self.create_matches_table = lambda: self.executescript(CREATE_MATCHES_TABLE)
        self.create_name_index = lambda: self.executescript(CREATE_NAME_INDEX)

    def add_camera(
        self, model, width, height, params, prior_focal_length=False, camera_id=None
    ):
        params = np.asarray(params, np.float64)
        cursor = self.execute(
            "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
            (
                camera_id,
                model,
                width,
                height,
                array_to_blob(params),
                prior_focal_length,
            ),
        )
        return cursor.lastrowid

    def add_image(
        self, name, camera_id, prior_q=np.zeros(4), prior_t=np.zeros(3), image_id=None
    ):
        cursor = self.execute(
            "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (
                image_id,
                name,
                camera_id,
                prior_q[0],
                prior_q[1],
                prior_q[2],
                prior_q[3],
                prior_t[0],
                prior_t[1],
                prior_t[2],
            ),
        )
        return cursor.lastrowid

    def add_keypoints(self, image_id, keypoints):
        assert len(keypoints.shape) == 2
        assert keypoints.shape[1] in [2, 4, 6]

        keypoints = np.asarray(keypoints, np.float32)
        self.execute(
            "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
            (image_id,) + keypoints.shape + (array_to_blob(keypoints),),
        )

    def add_descriptors(self, image_id, descriptors):
        descriptors = np.ascontiguousarray(descriptors, np.uint8)
        self.execute(
            "INSERT INTO descriptors VALUES (?, ?, ?, ?)",
            (image_id,) + descriptors.shape + (array_to_blob(descriptors),),
        )

    def add_matches(self, image_id1, image_id2, matches):
        assert len(matches.shape) == 2
        assert matches.shape[1] == 2

        if image_id1 > image_id2:
            matches = matches[:, ::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        self.execute(
            "INSERT INTO matches VALUES (?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches),),
        )

    def add_two_view_geometry(
        self,
        image_id1,
        image_id2,
        matches,
        F=np.eye(3),
        E=np.eye(3),
        H=np.eye(3),
        config=2,
    ):
        assert len(matches.shape) == 2
        assert matches.shape[1] == 2

        if image_id1 > image_id2:
            matches = matches[:, ::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        F = np.asarray(F, dtype=np.float64)
        E = np.asarray(E, dtype=np.float64)
        H = np.asarray(H, dtype=np.float64)
        self.execute(
            "INSERT INTO two_view_geometries VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
            (pair_id,)
            + matches.shape
            + (
                array_to_blob(matches),
                config,
                array_to_blob(F),
                array_to_blob(E),
                array_to_blob(H),
            ),
        )

# DB operation

In [108]:
# Modified from https://github.com/cvlab-epfl/disk/blob/37f1f7e971cea3055bb5ccfc4cf28bfd643fa339/colmap/h5_to_db.py

#  Copyright [2020] [Michał Tyszkiewicz, Pascal Fua, Eduard Trulls]
#
#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at
#
#       http://www.apache.org/licenses/LICENSE-2.0
#
#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.

import os, argparse, h5py, warnings
import numpy as np
from tqdm import tqdm
from PIL import Image, ExifTags


def get_focal(image_path, err_on_default=False):
    image = Image.open(image_path)
    max_size = max(image.size)

    exif = image.getexif()
    
    #
    # Modified to add exif_ifd to exif dict
    #
    exif_ifd = exif.get_ifd(0x8769)
    exif.update(exif_ifd)

    focal = None
    is_from_exif = False
    if exif is not None:
        focal_35mm = None
        # https://github.com/colmap/colmap/blob/d3a29e203ab69e91eda938d6e56e1c7339d62a99/src/util/bitmap.cc#L299
        for tag, value in exif.items():
            focal_35mm = None
            if ExifTags.TAGS.get(tag, None) == "FocalLengthIn35mmFilm":
                focal_35mm = float(value)
                is_from_exif = True
                break

        if focal_35mm is not None:
            focal = focal_35mm / 35.0 * max_size

    if focal is None:
        if err_on_default:
            raise RuntimeError("Failed to find focal length")

        # failed to find it in exif, use prior
        FOCAL_PRIOR = 1.2
        focal = FOCAL_PRIOR * max_size
    
    #
    # Modified to return a bool indicating if the focal length is from exif 
    #
    return focal, is_from_exif


def create_camera(db, image_path, camera_model):
    image = Image.open(image_path)
    width, height = image.size

    focal, is_from_exif = get_focal(image_path)

    if camera_model == "simple-pinhole":
        model = 0  # simple pinhole
        param_arr = np.array([focal, width / 2, height / 2])
    if camera_model == "pinhole":
        model = 1  # pinhole
        param_arr = np.array([focal, focal, width / 2, height / 2])
    elif camera_model == "simple-radial":
        model = 2  # simple radial
        param_arr = np.array([focal, width / 2, height / 2, 0.1])
    elif camera_model == "opencv":
        model = 4  # opencv
        param_arr = np.array([focal, focal, width / 2, height / 2, 0.0, 0.0, 0.0, 0.0])

    #
    # Modified to set prior_focal_length if the focal length is from exif
    #
    return db.add_camera(
        model, width, height, param_arr, prior_focal_length=is_from_exif
    )


def add_kpts_matches(db, img_dir, kpts, matches, fms = None):
    fname_to_id = {}

    # Add keypoints
    for filename in tqdm(kpts):
        path = os.path.join(img_dir, filename)
        camera_model = "simple-radial"
        camera_id = create_camera(db, path, camera_model)
        image_id = db.add_image(filename, camera_id)
        fname_to_id[filename] = image_id
        db.add_keypoints(image_id, kpts[filename])

        n_keys = len(matches)
        n_total = (n_keys * (n_keys - 1)) // 2
    # Add matches
    added = set()
    with tqdm(total=n_total) as pbar:
        for key1 in matches:
            for key2 in matches[key1]:
                id_1 = fname_to_id[key1]
                id_2 = fname_to_id[key2]
                pair_id = image_ids_to_pair_id(id_1, id_2)
                if pair_id in added:
                    warnings.warn(f"Pair {pair_id} ({id_1}, {id_2}) already added!")
                    continue
                db.add_matches(id_1, id_2, matches[key1][key2])
                added.add(pair_id)
                pbar.update(1)
                if fms is not None:
                    db.add_two_view_geometry(id_1, id_2, matches[key1][key2], fms[key1][key2], np.eye(3), np.eye(3))

    db.commit()


In [109]:
def get_unique_idxs(A, dim=0):
    # https://stackoverflow.com/questions/72001505/how-to-get-unique-elements-and-their-firstly-appeared-indices-of-a-pytorch-tenso
    unique, idx, counts = torch.unique(
        A, dim=dim, sorted=True, return_inverse=True, return_counts=True
    )
    _, ind_sorted = torch.sort(idx, stable=True)
    cum_sum = counts.cumsum(0)
    cum_sum = torch.cat((torch.tensor([0], device=cum_sum.device), cum_sum[:-1]))
    first_indices = ind_sorted[cum_sum]
    return first_indices

# AffNetHardNet Models

In [110]:
# Making kornia local features loading w/o internet
class AffNetHardNet(KF.LocalFeature):
    """Convenience module, which implements KeyNet detector + AffNet + HardNet descriptor.

    .. image:: _static/img/keynet_affnet.jpg
    """

    def __init__(
        self,
        num_features: int = 5000,
        upright: bool = False,
        device=torch.device("cpu"),
        scale_laf: float = 1.0,
        detector = "keynet"
    ):
        detector_options = ["keynet", "GFTT", "Hessian", "Harris", "DoG"]
        if detector not in detector_options:
            raise ValueError("Detector must be one of {}".format(detector_options))
        
        ori_module = (
            KF.PassLAF()
            if upright
            else KF.LAFOrienter(angle_detector=KF.OriNet(False)).eval()
        )
        if not upright:
            weights = torch.load(os.path.join(MODEL_DIR, "OriNet.pth"))["state_dict"]
            ori_module.angle_detector.load_state_dict(weights)

        config = {
            # Extraction Parameters
            "nms_size": 15,
            "pyramid_levels": 4,
            "up_levels": 1,
            "scale_factor_levels": math.sqrt(2),
            "s_mult": 22.0,
        }

        if detector == "keynet":
            detector = KF.KeyNetDetector(
            False,
            num_features=num_features,
            ori_module=ori_module,
            aff_module=KF.LAFAffNetShapeEstimator(False).eval(),
            ).to(device)
            kn_weights = torch.load(os.path.join(MODEL_DIR, "keynet_pytorch.pth"))[
            "state_dict"
            ]
            detector.model.load_state_dict(kn_weights)
        elif detector == "GFTT":
            detector = KF.MultiResolutionDetector(
                KF.CornerGFTT(),
                num_features=num_features,
#                 config=config,
                ori_module=ori_module,
                aff_module=KF.LAFAffNetShapeEstimator(False).eval(),
            ).to(device)
        elif detector == "Harris":
            detector = KF.MultiResolutionDetector(
                KF.CornerHarris(0.04),
                num_features=num_features,
#                 config=config,
                ori_module=ori_module,
                aff_module=KF.LAFAffNetShapeEstimator(False).eval(),
            ).to(device)
        elif detector == "DoG":
            detector = KF.MultiResolutionDetector(
                KF.BlobDoGSingle(),
                num_features=num_features,
#                 config=config,
                ori_module=ori_module,
                aff_module=KF.LAFAffNetShapeEstimator(False).eval(),
            ).to(device)
        affnet_weights = torch.load(os.path.join(MODEL_DIR, "AffNet.pth"))["state_dict"]
        detector.aff.load_state_dict(affnet_weights)

        # hardnet = KF.HardNet(False).eval()
        # hn_weights = torch.load(os.path.join(MODEL_DIR, "HardNetLib.pth"))["state_dict"]
        # hardnet.load_state_dict(hn_weights)
        # descriptor = KF.LAFDescriptor(
        #     hardnet, patch_size=32, grayscale_descriptor=True
        # ).to(device)
        hardnet8 = KF.HardNet8(False).eval()
        hn8_weights = torch.load(HARDNET_PT)
        hardnet8.load_state_dict(hn8_weights)
        descriptor = KF.LAFDescriptor(
            hardnet8, patch_size=32, grayscale_descriptor=True
        ).to(device)
        super().__init__(detector, descriptor, scale_laf)


In [111]:
def get_unique_matches(f_match_kpts):
    kpts = defaultdict(list)
    match_indexes = defaultdict(dict)
    total_kpts = defaultdict(int)
    for key1 in f_match_kpts:
        for key2 in f_match_kpts[key1]:
            matches = f_match_kpts[key1][key2]
            kpts[key1].append(matches[:, :2])
            kpts[key2].append(matches[:, 2:])
            current_match = torch.arange(len(matches)).reshape(-1, 1).repeat(1, 2)
            current_match[:, 0] += total_kpts[key1]
            current_match[:, 1] += total_kpts[key2]
            total_kpts[key1] += len(matches)
            total_kpts[key2] += len(matches)
            match_indexes[key1][key2] = current_match

    for key in kpts:
        kpts[key] = np.round(np.concatenate(kpts[key], axis=0))

    unique_kpts = {}
    unique_match_idxs = {}
    out_match = defaultdict(dict)

    for key in kpts.keys():
        uniq_kps, uniq_reverse_idxs = torch.unique(
            torch.from_numpy(kpts[key]), dim=0, return_inverse=True
        )
        unique_match_idxs[key] = uniq_reverse_idxs
        unique_kpts[key] = uniq_kps.numpy()

    for key1 in match_indexes:
        for key2 in match_indexes[key1]:
            m2 = deepcopy(match_indexes[key1][key2])
            m2[:, 0] = unique_match_idxs[key1][m2[:, 0]]
            m2[:, 1] = unique_match_idxs[key2][m2[:, 1]]
            mkpts = np.concatenate(
                [
                    unique_kpts[key1][m2[:, 0]],
                    unique_kpts[key2][m2[:, 1]],
                ],
                axis=1,
            )
            unique_idxs_current = get_unique_idxs(torch.from_numpy(mkpts), dim=0)
            m2_semiclean = m2[unique_idxs_current]
            unique_idxs_current1 = get_unique_idxs(m2_semiclean[:, 0], dim=0)
            m2_semiclean = m2_semiclean[unique_idxs_current1]
            unique_idxs_current2 = get_unique_idxs(m2_semiclean[:, 1], dim=0)
            m2_semiclean2 = m2_semiclean[unique_idxs_current2]
            out_match[key1][key2] = m2_semiclean2.numpy()
    return unique_kpts, out_match

# Scene feature detector

In [112]:
class AffNetHardNetDetector:
    def __init__(
        self,
        model,
        device=torch.device("cuda"),
        resize_long_edge_to=600,
        matcher="adalam",
        min_matches=15,
        rgb_input = False
    ):
        self.rgb_input = rgb_input
        print("Init AffNetHardNetDetector")
        self.model = model
        self.device = device
        self.resize_long_edge_to = resize_long_edge_to
        print("Longer edge will be resized to", self.resize_long_edge_to)

    def detect_features(self, img_fnames):
        f_lafs = dict()
        f_descs = dict()
        f_kpts = dict()
        f_raw_size = dict()
        f_matches = dict()
        # Get features
        print("Detecting AffNetHardNet features")
        for img_path in tqdm(img_fnames):
            img_fname = img_path.split("/")[-1]
            key = img_fname
            f_matches[key] = dict()
            with torch.inference_mode():
                timg = load_torch_image(img_path, device=device)
                raw_size = torch.tensor(timg.shape[2:])
                timg_resized, h_scale, w_scale = resize_torch_image(
                    timg, self.resize_long_edge_to, disable_enlarge=True
                )
                if self.rgb_input:
                    lafs, resps, descs = self.model(timg_resized)
                else:
                    lafs, resps, descs = self.model(K.color.rgb_to_grayscale(timg_resized))
                
                # Recover scale?
                lafs[:, :, 0, :] *= 1 / w_scale
                lafs[:, :, 1, :] *= 1 / h_scale
                desc_dim = descs.shape[-1]
                # Move keypoints to cpu for later colmap operations
                kpts = KF.get_laf_center(lafs).reshape(-1, 2).detach().cpu().numpy()
                descs = descs.reshape(-1, desc_dim).detach()
                f_lafs[key] = lafs.detach()
                f_kpts[key] = kpts
                f_descs[key] = descs
                f_raw_size[key] = raw_size
        gc.collect()
        torch.cuda.empty_cache()
        return f_lafs, f_kpts, f_descs, f_raw_size



## Scene LAF matcher

In [113]:
class LafMatcher:
    def __init__(self, min_matches=15, device="cuda", matcher="adalam"):
        self.adalam_config = KF.adalam.get_adalam_default_config()
        self.adalam_config["force_seed_mnn"] = True
        self.adalam_config["search_expansion"] = 16
        self.adalam_config["ransac_iters"] = 256
        self.adalam_config["device"] = device
        # self.adalam_config["orientation_difference_threshold"] = None
        # self.adalam_config['scale_rate_threshold'] = None
        self.min_matches = min_matches
        self.matcher = matcher

    def match(self, img_fnames, f_lafs, f_kpts, f_descs, f_raw_size, get_roi = False):
        index_pairs = dict()
        num_imgs = len(img_fnames)
        print("Matching to get index pairs")
        pair_count = 0
        f_matches = defaultdict(dict)
        f_rois = defaultdict(dict)
        for idx1 in tqdm(range(num_imgs - 1)):
            index_pairs[idx1] = []
            for idx2 in range(idx1 + 1, num_imgs):
                fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
                key1, key2 = fname1.split("/")[-1], fname2.split("/")[-1]
                lafs1 = f_lafs[key1]
                lafs2 = f_lafs[key2]
                desc1 = f_descs[key1]
                desc2 = f_descs[key2]
                if self.matcher == "adalam":
                    hw1, hw2 = f_raw_size[key1], f_raw_size[key2]

                    dists, idxs = KF.match_adalam(
                        desc1,
                        desc2,
                        lafs1,
                        lafs2,  # Adalam takes into account also geometric information
                        hw1=hw1,
                        hw2=hw2,
                        config=self.adalam_config,
                    )  # Adalam also benefits from knowing image size
                else:
                    dists, idxs = KF.match_smnn(desc1, desc2, 0.98)

                if dists.mean().cpu().numpy() < 0.5:
                    first_indices = get_unique_idxs(idxs[:, 1])
                    idxs = idxs[first_indices]
                    dists = dists[first_indices]
                    n_matches = len(idxs)
                    if n_matches >= self.min_matches:
                        pair_count += 1
                        index_pairs[idx1].append(
                            [idx2, dists.mean().cpu().numpy().item(), n_matches]
                        )
                        f_matches[key1][key2] = (
                            idxs.detach().cpu().numpy().reshape(-1, 2)
                        )

                        # Compute ROI
                        if get_roi:
                            mkpts1 = f_kpts[key1][idxs.cpu().numpy()[:, 0]]
                            mkpts2 = f_kpts[key2][idxs.cpu().numpy()[:, 1]]
                            roi_min_w_1, roi_max_w_1 = np.percentile(mkpts1[:, 0], [5, 95])
                            roi_min_h_1, roi_max_h_1 = np.percentile(mkpts1[:, 1], [5, 95])
                            roi_area_1 = (roi_max_w_1 - roi_min_w_1) * (
                                roi_max_h_1 - roi_min_h_1
                            )
                            roi1 = {
                                "roi_min_w": roi_min_w_1,
                                "roi_min_h": roi_min_h_1,
                                "roi_max_w": roi_max_w_1,
                                "roi_max_h": roi_max_h_1,
                                "area": roi_area_1,
                            }
                            roi_min_w_2, roi_max_w_2 = np.percentile(mkpts2[:, 0], [5, 95])
                            roi_min_h_2, roi_max_h_2 = np.percentile(mkpts2[:, 1], [5, 95])
                            roi_area_2 = (roi_max_w_2 - roi_min_w_2) * (
                                roi_max_h_2 - roi_min_h_2
                            )
                            roi2 = {
                                "roi_min_w": roi_min_w_2,
                                "roi_min_h": roi_min_h_2,
                                "roi_max_w": roi_max_w_2,
                                "roi_max_h": roi_max_h_2,
                                "area": roi_area_2,
                            }
                            f_rois[key1][key2] = [roi1, roi2]

        print(f" Get {pair_count} from {int(num_imgs * (num_imgs-1)/2)} possible pairs")
        torch.cuda.empty_cache()
        gc.collect()
        return index_pairs, f_kpts, f_matches, f_rois

# Matches and pairs operation

In [114]:
def merge_kpts_matches(kpts, matches, new_kpts, new_matches, cap = None):
    # merge kpts
    prev_len = dict()
    for new_key in new_kpts:
        if new_key in kpts:
            old_len = len(kpts[new_key])
            kpts[new_key] = np.concatenate([kpts[new_key], new_kpts[new_key]], axis=0)
        else:
            old_len = 0
            kpts[new_key] = new_kpts[new_key]
        prev_len[new_key] = old_len

    for new_key1 in new_matches:
        for new_key2 in new_matches[new_key1]:
            old_len1 = prev_len[new_key1]
            old_len2 = prev_len[new_key2]
            new_match = new_matches[new_key1][new_key2] + [old_len1, old_len2]
            if cap is not None and len(new_match) > cap:
                keep = np.random.choice(len(new_match), cap, replace=False)
                new_match = new_match[keep, :]
            if new_key1 in matches and new_key2 in matches[new_key1]:

                matches[new_key1][new_key2] = np.concatenate(
                    [
                        matches[new_key1][new_key2],
                        new_match,
                    ],
                    axis=0,
                )
            else:
                if new_key1 not in matches:
                    matches[new_key1] = dict()
                matches[new_key1][new_key2] = new_match
    return kpts, matches


def keep_matches(matches, max_num=None):
    if max_num is None:
        return matches
    if len(matches) > max_num:
        # radnomly select max_num matches
        matches = np.random.choice(matches, max_num, replace=False)
    return matches


def keep_pairs(index_pairs, max_num_pairs=20):
    new_count = 0
    old_count = 0
    new_idx_count = defaultdict(int)
    new_pairs = defaultdict(list)
    for key1 in index_pairs:
        # sort pairs by number of pairs
        index_pairs[key1] = sorted(index_pairs[key1], key=lambda x: x[2], reverse=True)
        for pair in index_pairs[key1]:
            old_count += 1
            idx1 = key1
            idx2 = pair[0]

            if new_idx_count[key1] < max_num_pairs:
                new_pairs[idx1].append(pair)
                new_count += 1
                new_idx_count[idx1] += 1
                new_idx_count[idx2] += 1
            else:
                continue

    if DEBUG:
        print(f"origin pairs: {old_count}, kept pairs: {new_count}")
    return index_pairs

def select_matches(matches, keep_ratio = 0.01):
    max_matches = defaultdict(int)
    old_matches_count = 0
    for key1 in matches:
        for key2 in matches[key1]:
            max_matches[key1] = max(max_matches[key1], len(matches[key1][key2]))
            max_matches[key2] = max(max_matches[key2], len(matches[key1][key2]))
            old_matches_count +=1

    new_matches_count = 0
    new_matches = defaultdict(dict)
    for key1 in matches:
        for key2 in matches[key1]:
            n_matches = len(matches[key1][key2])
            if n_matches > max_matches[key1] * keep_ratio or n_matches > max_matches[key2] * keep_ratio:
                new_matches[key1][key2] = matches[key1][key2]
                new_matches_count+=1
    if DEBUG:
        print(f"origin matches: {old_matches_count}, kept matches: {new_matches_count}")
    return new_matches

# Get Fundamental matrices from matches and keypoints

In [115]:
def get_fms(kpts, matches):
    prev_len = dict()
    fms = defaultdict(dict)
    print("Get Fundamental Matrix")
    for key1 in tqdm(matches):
        for key2 in matches[key1]:
            match = matches[key1][key2]
            mkpts1 = kpts[key1][match[:, 0]]
            mkpts2 = kpts[key2][match[:, 1]]
            Fm, inliers = cv2.findFundamentalMat(mkpts1, mkpts2, cv2.USAC_ACCURATE, FM_PARAMS["ransacReprojThreshold"], FM_PARAMS["confidence"], FM_PARAMS["maxIters"])
            #keep inliers matches
            #print how many matches are inliers
            # print(f"key1: {key1}, key2: {key2}, inliers: {len(new_match)}/{len(match)}")
            if FM_PARAMS["removeOutliers"] == True:
                new_match = match[inliers.ravel() == 1]
                matches[key1][key2] = new_match
            fms[key1][key2] = Fm
    # print(Fm.shape)
    return kpts, matches, fms

# Model Setup

In [116]:
if MODEL_DICT["Keynet"]["enable"]:
    keynet_model = (
        AffNetHardNet(num_features=12000, upright=False, device=device, detector="keynet")
        .to(device)
        .eval()
    )
    keynet_detector = AffNetHardNetDetector(keynet_model, resize_long_edge_to=MODEL_DICT["Keynet"]["resize_long_edge_to"])
    laf_matcher = LafMatcher(device=device)
    
if MODEL_DICT["GFTT"]["enable"]:
    gftt_model = (
        AffNetHardNet(num_features=12000, upright=False, device=device, detector="GFTT")
        .to(device)
        .eval()
    )
    gftt_detector = AffNetHardNetDetector(gftt_model, resize_long_edge_to=MODEL_DICT["GFTT"]["resize_long_edge_to"])
    laf_matcher = LafMatcher(device=device)

if MODEL_DICT["DoG"]["enable"]:
    DoG_model = (
        AffNetHardNet(num_features=12000, upright=False, device=device, detector="DoG")
        .to(device)
        .eval()
    )
    DoG_detector = AffNetHardNetDetector(DoG_model, resize_long_edge_to=MODEL_DICT["DoG"]["resize_long_edge_to"])
    laf_matcher = LafMatcher(device=device)

if MODEL_DICT["Harris"]["enable"]:
    harris_model = (
        AffNetHardNet(num_features=12000, upright=False, device=device, detector="Harris")
        .to(device)
        .eval()
    )
    harris_detector = AffNetHardNetDetector(harris_model, resize_long_edge_to=MODEL_DICT["Harris"]["resize_long_edge_to"])
    laf_matcher = LafMatcher(device=device)

/tmp/ipykernel_76/3545578030.py:43: DeprecationWarning: `LAFAffNetShapeEstimator` default behaviour is changed and now it does preserve original LAF orientation. Make sure your code accounts for this.
  aff_module=KF.LAFAffNetShapeEstimator(False).eval(),


Init AffNetHardNetDetector
Longer edge will be resized to 1600


/tmp/ipykernel_76/3545578030.py:55: DeprecationWarning: `LAFAffNetShapeEstimator` default behaviour is changed and now it does preserve original LAF orientation. Make sure your code accounts for this.
  aff_module=KF.LAFAffNetShapeEstimator(False).eval(),


Init AffNetHardNetDetector
Longer edge will be resized to 1600


/tmp/ipykernel_76/3545578030.py:71: DeprecationWarning: `LAFAffNetShapeEstimator` default behaviour is changed and now it does preserve original LAF orientation. Make sure your code accounts for this.
  aff_module=KF.LAFAffNetShapeEstimator(False).eval(),


Init AffNetHardNetDetector
Longer edge will be resized to 1600


/tmp/ipykernel_76/3545578030.py:63: DeprecationWarning: `LAFAffNetShapeEstimator` default behaviour is changed and now it does preserve original LAF orientation. Make sure your code accounts for this.
  aff_module=KF.LAFAffNetShapeEstimator(False).eval(),


Init AffNetHardNetDetector
Longer edge will be resized to 1600


In [117]:
# Util to check if the pairs are identital
def compare_pairs(pairs1, pairs2):
    pair1_dict = dict()
    pair2_dict = dict()
    for idx1 in tqdm(range(len(pairs1) - 1)):
        for pair in pairs1[idx1]:
            pair1_dict[(idx1, pair[0])] = 0
    for idx2 in tqdm(range(len(pairs2) - 1)):
        for pair in pairs2[idx2]:
            pair2_dict[(idx2, pair[0])] = 0

    for key in tqdm(pair1_dict):
        if key not in pair2_dict:
            print(f"Key{key} not in pair2_dict")
    for key in tqdm(pair2_dict):
        if key not in pair1_dict:
            print(f"Key{key} not in pair1_dict")

# Function to generate scene db for reconstruction

In [118]:
def generate_scene_db(dataset, scene):
    feature_det_start = time()
    # Process a scene and write matches and keypoints to the database
    img_dir = f"{SRC}/{MODE}/{dataset}/images"
    if not os.path.exists(img_dir):
        print("Image dir does not exist:", img_dir)
        return
    
    img_fnames = [f"{SRC}/{x}" for x in data_dict[dataset][scene]]
    print(f"Got {len(img_fnames)} images")

    matches = dict()
    kpts = dict()

    if MODEL_DICT["Keynet"]["enable"]:
        f_lafs, f_kpts, f_descs, f_raw_size = keynet_detector.detect_features(
            img_fnames
        )
        keynet_pairs, keynet_kpts, keynet_matches, keynet_rois = laf_matcher.match(
            img_fnames, f_lafs, f_kpts, f_descs, f_raw_size
        )
        if not MODEL_DICT["Keynet"]["pair_only"]:
            kpts, matches = merge_kpts_matches(kpts, matches, keynet_kpts, keynet_matches, MATCHES_CAP)

    if MODEL_DICT["GFTT"]["enable"]:
        gftt_lafs, gftt_kpts, gftt_descs, gftt_raw_size = gftt_detector.detect_features(
            img_fnames
        )
        index_pairs, gftt_kpts, gftt_matches, gftt_rois = laf_matcher.match(
            img_fnames, gftt_lafs, gftt_kpts, gftt_descs, gftt_raw_size
        )
        kpts, matches = merge_kpts_matches(kpts, matches, gftt_kpts, gftt_matches, MATCHES_CAP)

    if MODEL_DICT["DoG"]["enable"]:
        DoG_lafs, DoG_kpts, DoG_descs, DoG_raw_size = DoG_detector.detect_features(
            img_fnames
        )
        index_pairs, DoG_kpts, DoG_matches, DoG_rois = laf_matcher.match(
            img_fnames, DoG_lafs, DoG_kpts, DoG_descs, DoG_raw_size
        )
        kpts, matches = merge_kpts_matches(kpts, matches, DoG_kpts, DoG_matches, MATCHES_CAP)
        
    if MODEL_DICT["Harris"]["enable"]:
        harris_lafs, harris_kpts, harris_descs, harris_raw_size = harris_detector.detect_features(
            img_fnames
        )
        harris_pairs, harris_kpts, harris_matches, harris_rois = laf_matcher.match(
            img_fnames, harris_lafs, harris_kpts, harris_descs, harris_raw_size
        )
        kpts, matches = merge_kpts_matches(kpts, matches, harris_kpts, harris_matches, MATCHES_CAP)
        # compare_pairs(keynet_pairs, harris_pairs)

    # Get fundamental matrices
    kpts, matches, fms = get_fms(kpts, matches)
    
    matches = select_matches(matches, MATCH_FILTER_RATIO)

    if DEBUG:
        import random
        random.seed(0)
        for i in range(5):
            print(matches.keys())
           
            key1 = random.choice(list(matches.keys()))
            key2 = random.choice(list(matches[key1].keys()))
            print(key1, key2)
            fname1, fname2 = os.path.join(img_dir, key1), os.path.join(img_dir, key2)

            print("Plot Combined matches")
            plot_images_with_keypoints(
                fname1, fname2, kpts[key1], kpts[key2], matches[key1][key2]
            )
    # Write to database
    feature_dir = f"featureout/{dataset}_{scene}"
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir, exist_ok=True)
    database_path = f"{feature_dir}/colmap.db"
    if os.path.isfile(database_path):
        os.remove(database_path)

    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    print("Add kpts and matches to database")
    add_kpts_matches(db, img_dir, kpts, matches, fms)
    feature_det_end = time()
    matching_time = feature_det_end - feature_det_start
    torch.cuda.empty_cache()
    gc.collect()

    return matching_time

# Function of reconstruction

In [119]:
def reconstruct_from_db(dataset, scene, init_img):
    scene_result = {}
    reconst_start = time()

    img_dir = f"{SRC}/{MODE}/{dataset}/images"
    if not os.path.exists(img_dir):
        print("Image dir does not exist:", img_dir)
        return

    feature_dir = f"featureout/{dataset}_{scene}"
    database_path = f"{feature_dir}/colmap.db"
    db = COLMAPDatabase.connect(database_path)
    output_path = f"{feature_dir}/colmap_rec"
    t = time()
    gc.collect()
    
    t = time() - t
    print(f"RANSAC in  {t:.4f} sec")
    t = time()

    # By default colmap does not generate a reconstruction if less than 10 images are registered. Lower it to 3.
    mapper_options = pycolmap.IncrementalPipelineOptions()
    mapper_options.init_image_id1 = init_img
    mapper_options.min_model_size = 5
    # mapper_options.ba_min_num_residuals_for_multi_threading = 120000
    # mapper_options.ba_global_max_num_iterations = 30
    # mapper_options.ba_local_max_num_iterations = 15
    

    for attribute_name in dir(mapper_options):
        if not attribute_name.startswith("__"):
            attribute_value = getattr(mapper_options, attribute_name)
            print(f"{attribute_name}: {attribute_value}")
    os.makedirs(output_path, exist_ok=True)
    
    maps = pycolmap.incremental_mapping(
        database_path=database_path,
        image_path=img_dir,
        output_path=output_path,
        options=mapper_options,
    )
    print(maps)
    t = time() - t
    print(f"Reconstruction done in  {t:.4f} sec")
    imgs_registered = 0
    best_idx = None
    print("Looking for the best reconstruction")
    if isinstance(maps, dict):
        for idx1, rec in maps.items():
            print(idx1, rec.summary())
            if len(rec.images) > imgs_registered:
                imgs_registered = len(rec.images)
                best_idx = idx1
    if best_idx is not None:
        print(maps[best_idx].summary())
        for k, im in maps[best_idx].images.items():
            key1 = f"{MODE}/{dataset}/images/{im.name}"
            scene_result[key1] = {}
            scene_result[key1]["R"] = deepcopy(im.cam_from_world.rotation.matrix())
            scene_result[key1]["t"] = deepcopy(np.array(im.cam_from_world.translation))

    gc.collect()
    reconst_end = time()
    reconst_time = reconst_end - reconst_start
    return scene_result, reconst_time


# Main Loop

In [122]:
import gc
from collections import defaultdict

def find_initial_image_id(db):
    image_stats = defaultdict(lambda: {"pairs": 0, "matches": 0})

    # Use a generator to fetch rows in chunks to avoid loading everything into memory at once
    cursor = db.execute("SELECT pair_id, data FROM matches")
    while True:
        rows = cursor.fetchmany(5)  # Fetch 10 rows at a time
        if not rows:
            break
        for row in rows:
            pair_id, data = row
            image_id1, image_id2 = pair_id_to_image_ids(pair_id)
            num_matches = len(blob_to_array(data, dtype=np.uint32, shape=(-1, 2)))

            image_stats[image_id1]["pairs"] += 1
            image_stats[image_id1]["matches"] += num_matches
            image_stats[image_id2]["pairs"] += 1
            image_stats[image_id2]["matches"] += num_matches
    
    cursor.close()  # Close cursor to free resources

    # Determine the initial image ID based on the highest number of pairs and matches
    initial_image_id = max(image_stats.items(), key=lambda item: (item[1]["pairs"], item[1]["matches"]))[0]

    # Free up memory
    del image_stats
    gc.collect()

    return initial_image_id


In [123]:
# Main loop to add kpts and matches
datasets = []
time_dict = dict()

for dataset in data_dict:
    datasets.append(dataset)

if DEBUG:
    matching_start = time()
    for dataset, scene in all_scenes:
        print(dataset, scene)
        time_dict["matching-" + scene] = generate_scene_db(dataset, scene)

    matching_end = time()
    time_dict["matching-TOTAL"] = matching_end - matching_start
else:
    # Run db generation and reconstuction with multiprocessing if not DEBUG
    out_results = defaultdict(dict)
    total_start = time()
    with concurrent.futures.ProcessPoolExecutor(max_workers=NUM_CORES) as executors:
        futures = defaultdict(dict)

        for dataset, scene in all_scenes:
            print(dataset, scene)
            time_dict["matching-" + scene] = generate_scene_db(dataset, scene)
            
            # Get initial image
            feature_dir = f"featureout/{dataset}_{dataset}"
            database_path = f"{feature_dir}/colmap.db"
            db = COLMAPDatabase.connect(database_path)
            init_img = find_initial_image_id(db)
            del db
            
            futures[dataset][scene] = executors.submit(reconstruct_from_db, dataset, scene, init_img)

        for dataset, scene in all_scenes:
            result = futures[dataset][scene].result()
            if result is not None:
                out_results[dataset][scene], time_dict["reconst-" + scene] = result
    total_end = time()
    time_dict["TOTAL"] = total_end - total_start
    


church church
Got 41 images
Detecting AffNetHardNet features


100%|██████████| 41/41 [00:17<00:00,  2.31it/s]


Matching to get index pairs


100%|██████████| 40/40 [00:16<00:00,  2.42it/s]


 Get 148 from 820 possible pairs
Detecting AffNetHardNet features


...

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.481633e+04    0.00e+00    7.59e+07   0.00e+00   0.00e+00  1.00e+04        0    1.70e-02    1.08e-01
   1  2.472716e+04    8.92e+01    1.12e+08   5.61e+00   9.99e-01  3.00e+04        1    5.50e-02    1.63e-01
   2  2.472598e+04    1.18e+00    9.73e+07   2.25e+00   1.09e+00  9.00e+04        1    3.97e-02    2.03e-01
   3  2.472585e+04    1.28e-01    8.26e+07   9.01e-01   1.16e+00  2.70e+05        1    3.71e-02    2.40e-01
   4  2.472584e+04    1.86e-02    1.30e+08   7.02e-01   1.12e+00  8.10e+05        1    4.01e-02    2.80e-01
   5  2.472583e+04    2.03e-03    6.14e+07   6.14e-01   1.15e+00  2.43e+06        1    3.81e-02    3.18e-01
   6  2.472583e+04    9.58e-05    5.45e+07   1.62e-01   9.50e-01  7.29e+06        1    3.82e-02    3.57e-01
   7  2.472583e+04    3.84e-06    6.08e+07   2.95e-02   4.27e-01  7.27e+06        1    4.08e-02    3.97e-01
   8  2.472583e+04   -3.61e-

I20240531 20:39:17.693451  4943 misc.cc:205] 
Bundle adjustment report
------------------------
I20240531 20:39:17.693529  4943 bundle_adjustment.cc:942] 
    Residuals : 72568
   Parameters : 34158
   Iterations : 24
         Time : 0.95529 [s]
 Initial cost : 0.584785 [px]
   Final cost : 0.583718 [px]
  Termination : Convergence

I20240531 20:39:17.731477  4943 incremental_mapper.cc:175] => Completed observations: 3
I20240531 20:39:17.738368  4943 incremental_mapper.cc:178] => Merged observations: 0
I20240531 20:39:17.744307  4943 incremental_mapper.cc:160] => Filtered observations: 4
I20240531 20:39:17.744359  4943 incremental_mapper.cc:119] => Changed observations: 0.000193
I20240531 20:39:17.744426  4943 incremental_mapper.cc:167] => Filtered images: 0
I20240531 20:39:17.744443  4943 misc.cc:198] 
Registering image #10 (30)
I20240531 20:39:17.744491  4943 incremental_mapper.cc:495] => Image sees 39 / 376 points
I20240531 20:39:17.751096  4943 incremental_mapper.cc:535] => Could n

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.470664e+04    0.00e+00    1.30e+08   0.00e+00   0.00e+00  1.00e+04        0    1.97e-02    1.13e-01
   1  2.469799e+04    8.65e+00    6.80e+07   5.17e+00   1.00e+00  3.00e+04        1    5.47e-02    1.68e-01
   2  2.469774e+04    2.46e-01    6.51e+07   5.67e+00   1.01e+00  9.00e+04        1    4.81e-02    2.16e-01
   3  2.469760e+04    1.40e-01    7.26e+07   7.15e+00   9.03e-01  1.88e+05        1    4.50e-02    2.61e-01
   4  2.469756e+04    4.75e-02    9.90e+07   4.10e+00   9.61e-01  5.65e+05        1    3.74e-02    2.99e-01
   5  2.469755e+04    4.75e-03    9.08e+07   1.37e+00   1.02e+00  1.69e+06        1    4.51e-02    3.44e-01
   6  2.469755e+04    1.47e-04    1.65e+08   1.76e-01   1.21e+00  5.08e+06        1    4.57e-02    3.90e-01
   7  2.469755e+04    1.09e-06    5.78e+07   3.49e-02   1.57e-01  3.84e+06        1    6.13e-02    4.51e-01
   8  2.469755e+04   -8.67e-

I20240531 20:39:19.184969  4943 misc.cc:205] 
Bundle adjustment report
------------------------
I20240531 20:39:19.185258  4943 bundle_adjustment.cc:942] 
    Residuals : 72564
   Parameters : 34155
   Iterations : 31
         Time : 1.36608 [s]
 Initial cost : 0.583507 [px]
   Final cost : 0.5834 [px]
  Termination : Convergence

I20240531 20:39:19.217940  4943 incremental_mapper.cc:175] => Completed observations: 0
I20240531 20:39:19.225965  4943 incremental_mapper.cc:178] => Merged observations: 0
I20240531 20:39:19.233764  4943 incremental_mapper.cc:160] => Filtered observations: 1
I20240531 20:39:19.233928  4943 incremental_mapper.cc:119] => Changed observations: 0.000028
I20240531 20:39:19.234067  4943 incremental_mapper.cc:167] => Filtered images: 0
I20240531 20:39:19.254894  4943 misc.cc:198] 
Finding good initial image pair
I20240531 20:39:19.340226  4943 incremental_mapper.cc:404] => No good initial image pair found.
I20240531 20:39:19.340562  4943 timer.cc:91] Elapsed time: 

{0: Reconstruction(num_reg_images=29, num_cameras=29, num_points3D=11310, num_observations=36281)}
Reconstruction done in  134.3592 sec
Looking for the best reconstruction
0 Reconstruction:
	num_reg_images = 29
	num_cameras = 29
	num_points3D = 11310
	num_observations = 36281
	mean_track_length = 3.20787
	mean_observations_per_image = 1251.07
	mean_reprojection_error = 0.844644
Reconstruction:
	num_reg_images = 29
	num_cameras = 29
	num_points3D = 11310
	num_observations = 36281
	mean_track_length = 3.20787
	mean_observations_per_image = 1251.07
	mean_reprojection_error = 0.844644


# Reconstruction Loop

In [125]:
# Main loop for reconstruction
if DEBUG:
    reconst_start = time()
    out_results = defaultdict(dict)
    with concurrent.futures.ProcessPoolExecutor(max_workers=NUM_CORES) as executors:
        futures = defaultdict(dict)
        for dataset, scene in all_scenes:
            futures[dataset][scene] = executors.submit(reconstruct_from_db, dataset, scene)
            print("Submitted", dataset, scene)
        for dataset, scene in all_scenes:
            result = futures[dataset][scene].result()
            if result is not None:
                out_results[dataset][scene], time_dict["reconst-" + scene] = result

    reconst_end = time()
    time_dict["reconst-TOTAL"] = reconst_end - reconst_start

In [126]:
def create_submission(out_results, data_dict):
    with open(f'submission.csv', 'w') as f:
        f.write('image_path,dataset,scene,rotation_matrix,translation_vector\n')
        for dataset in data_dict:
            if dataset in out_results:
                res = out_results[dataset]
            else:
                res = {}
            
            for scene in data_dict[dataset]:
                if scene in res:
                    scene_res = res[scene]
                else:
                    scene_res = {"R":{}, "t":{}}
                
                for image in data_dict[dataset][scene]:
                    if image in scene_res:
                        print (image)
                        R = scene_res[image]['R'].reshape(-1)
                        T = scene_res[image]['t'].reshape(-1)
                    else:
                        R = np.eye(3).reshape(-1)
                        T = np.zeros((3))
                    f.write(f'{image},{dataset},{scene},{arr_to_str(R)},{arr_to_str(T)}\n')

In [127]:
create_submission(out_results, data_dict)

test/church/images/00090.png
test/church/images/00092.png
test/church/images/00050.png
test/church/images/00068.png
test/church/images/00083.png
test/church/images/00096.png
test/church/images/00069.png
test/church/images/00018.png
test/church/images/00030.png
test/church/images/00024.png
test/church/images/00032.png
test/church/images/00026.png
test/church/images/00008.png
test/church/images/00035.png
test/church/images/00021.png
test/church/images/00010.png
test/church/images/00039.png
test/church/images/00011.png
test/church/images/00013.png
test/church/images/00006.png
test/church/images/00012.png
test/church/images/00029.png
test/church/images/00001.png
test/church/images/00072.png
test/church/images/00066.png
test/church/images/00061.png
test/church/images/00074.png
test/church/images/00076.png
test/church/images/00063.png


In [128]:
import pandas as pd
pd.read_csv('submission.csv').sort_values('image_path').to_csv('sub_sorted.csv',index=False)
pd.read_csv('submission.csv').sort_values('image_path')

,image_path,dataset,scene,rotation_matrix,translation_vector
27,test/church/images/00001.png,church,church,0.9675649929310469;0.1176589534987946;0.223549...,0.742540307159214;1.4770941857459092;4.3725047...
24,test/church/images/00006.png,church,church,0.9865310174302264;-0.06146030354744919;0.1515...,-0.203802802012917;0.898835092502265;4.9300421...
17,test/church/images/00008.png,church,church,0.9978743825135169;-0.006451394674260986;-0.06...,-1.0609740279392559;1.2208647056994262;4.36606...
20,test/church/images/00010.png,church,church,0.9891888068842288;-0.028125019557057325;-0.14...,-0.7586859043129818;0.8562230029979684;4.24484...
22,test/church/images/00011.png,church,church,0.9986349748046586;0.0072432910997729215;-0.05...,0.017686211889582148;0.9728406155838472;3.3715...
25,test/church/images/00012.png,church,church,0.992242703978321;-0.0014339406407473197;-0.12...,-0.0024346979544574688;0.4047128553910488;1.90...
23,test/church/images/00013.png,church,church,0.9995140403556352;0.019530838838443264;0.0242...,0.28490256510265444;0.553371618824661;3.458314...
11,test/church/images/00018.png,church,church,0.9668012391390757;-0.07553772626127703;-0.244...,0.16295648949331237;0.8851280675611573;1.96509...
19,test/church/images/00021.png,church,church,0.976182125966181;0.16989966195246775;0.134916...,-2.2334692638767404;0.5509274018790263;1.27628...
13,test/church/images/00024.png,church,church,0.8398067780833977;0.23529642200442533;0.48924...,-0.7608992705827207;0.46370441390368955;1.3490...


In [76]:
d = pd.read_csv('image-matching-challenge-2024/train/train_labels.csv')
d = d[d['dataset'] == 'church']
d = d[d['image_name'].isin(os.listdir('image-matching-challenge-2024/test/church/images'))]
d.sort_values('image_name')

,image_name,rotation_matrix,translation_vector,calibration_matrix,dataset,scene
2050,00001.png,-0.13287735089213104;-0.38610781630581703;-0.9...,44.99988140229224;23.519592516025515;64.240283...,688.1327008743596;0.0;512.0;0.0;688.1327008743...,church,church
2060,00006.png,-0.27339640369488727;-0.4960815725341004;-0.82...,31.80778745504346;20.23740280732015;71.9083826...,738.2896550144395;0.0;384.0;0.0;738.2896550144...,church,church
2064,00008.png,-0.44971357625437647;-0.33623555290802987;-0.8...,17.23602659920966;23.102044054517226;75.072330...,2164.6993031995917;0.0;384.0;0.0;2164.69930319...,church,church
2068,00010.png,-0.5199196021099126;-0.30985522224198414;-0.79...,16.564406474748694;17.47344165400385;75.436237...,1360.9282397460088;0.0;384.0;0.0;1360.92823974...,church,church
2070,00011.png,-0.422747076932537;-0.338330246781646;-0.84072...,27.773105439252546;21.02019717121849;67.097448...,2209.8841784185324;0.0;384.0;0.0;2209.88417841...,church,church
2072,00012.png,-0.48483417582157906;-0.30646821686858705;-0.8...,24.60584432699671;13.414661500703362;57.924488...,2237.8220923527538;0.0;384.0;0.0;2237.82209235...,church,church
2074,00013.png,-0.35123404464437735;-0.3695096832227965;-0.86...,33.20290669350995;12.341836955071395;66.892628...,2215.998222158956;0.0;384.0;0.0;2215.998222158...,church,church
2001,00018.png,-0.6018537100515358;-0.2936475757196423;-0.742...,19.945332333212107;27.905416700537035;57.27640...,2221.2037023762787;0.0;384.0;0.0;2221.20370237...,church,church
2004,00021.png,-0.2350573521076662;-0.31318134763313876;-0.92...,15.653328089785836;15.925919308949593;42.44823...,741.0445473439818;0.0;384.0;0.0;741.0445473439...,church,church
2007,00024.png,0.16040734290359437;-0.37042253909724165;-0.91...,39.352419363363325;8.660353454832865;30.034651...,1408.2245532435304;0.0;384.0;0.0;1408.22455324...,church,church


In [88]:
import pandas as pd
pd.read_csv('submission.csv')

,image_path,dataset,scene,rotation_matrix,translation_vector
0,test/church/images/00046.png,church,church,0.5289866360414176;-0.042283698090655686;-0.84...,4.618137806451945;1.5704687127127797;-1.679641...
1,test/church/images/00090.png,church,church,0.9948224580235087;0.03947558301429717;-0.0936...,-0.10686961235197619;-0.09479320056956902;-0.9...
2,test/church/images/00092.png,church,church,0.9312592203696662;-0.07439939170193252;-0.356...,0.1531188565198357;-0.26159078674468345;-0.810...
3,test/church/images/00087.png,church,church,0.8692072469457504;-0.11261098742607238;-0.481...,0.8697852288205649;-0.21205437426363757;-0.825...
4,test/church/images/00050.png,church,church,0.9330326980076884;0.2088204592204329;0.292991...,-2.9042695886414007;0.2360630294279655;0.46016...
5,test/church/images/00068.png,church,church,0.5285011966991933;-0.16412000848041422;-0.832...,4.6153290603880475;0.0362059471405356;0.363501...
6,test/church/images/00083.png,church,church,0.913691338438969;-0.09799018030761318;-0.3944...,1.4236348244613546;-0.28409367578281464;-0.895...
7,test/church/images/00096.png,church,church,0.9597012261509148;0.12194565907305677;0.25318...,-1.1535711193981304;-0.09576434289143829;-1.31...
8,test/church/images/00069.png,church,church,0.5436305666650827;-0.1612463878811002;-0.8236...,4.622155709964973;-0.07990214866884832;0.23811...
9,test/church/images/00081.png,church,church,0.8608649386868216;-0.08667059256922169;-0.501...,1.5219209228276676;-0.2228853064466324;-0.6807...


# Evaluation Code
from https://www.kaggle.com/code/eduardtrulls/imc2023-evaluation

In [78]:
'''Image Matching Challenge 2024 - Hexathlon | Metric Example Notebook'''

import math
import numpy as np
import pandas as pd
import pandas.api.types

_EPS = np.finfo(float).eps * 4.0

# mAA evaluation thresholds per scene, different accoring to the scene
translation_thresholds_meters_dict = {
 'multi-temporal-temple-baalshamin':  np.array([0.025,  0.05,  0.1,  0.2,  0.5,  1.0]),
 'pond':                              np.array([0.025,  0.05,  0.1,  0.2,  0.5,  1.0]),
 'transp_obj_glass_cylinder':         np.array([0.0025, 0.005, 0.01, 0.02, 0.05, 0.1]),
 'transp_obj_glass_cup':              np.array([0.0025, 0.005, 0.01, 0.02, 0.05, 0.1]),
 'church':                            np.array([0.025,  0.05,  0.1,  0.2,  0.5,  1.0]),
 'lizard':                            np.array([0.025,  0.05,  0.1,  0.2,  0.5,  1.0]),
 'dioscuri':                          np.array([0.025,  0.05,  0.1,  0.2,  0.5,  1.0]), 
}


def vector_norm(data, axis=None, out=None):
    '''Return length, i.e. Euclidean norm, of ndarray along axis.'''
    data = np.array(data, dtype=np.float64, copy=True)
    if out is None:
        if data.ndim == 1:
            return math.sqrt(np.dot(data, data))
        data *= data
        out = np.atleast_1d(np.sum(data, axis=axis))
        np.sqrt(out, out)
        return out
    data *= data
    np.sum(data, axis=axis, out=out)
    np.sqrt(out, out)
    return None


def quaternion_matrix(quaternion):
    '''Return homogeneous rotation matrix from quaternion.'''
    q = np.array(quaternion, dtype=np.float64, copy=True)
    n = np.dot(q, q)
    if n < _EPS:
        # print("special case")
        return np.identity(4)
    q *= math.sqrt(2.0 / n)
    q = np.outer(q, q)
    return np.array(
        [
            [
                1.0 - q[2, 2] - q[3, 3],
                q[1, 2] - q[3, 0],
                q[1, 3] + q[2, 0],
                0.0,
            ],
            [
                q[1, 2] + q[3, 0],
                1.0 - q[1, 1] - q[3, 3],
                q[2, 3] - q[1, 0],
                0.0,
            ],
            [
                q[1, 3] - q[2, 0],
                q[2, 3] + q[1, 0],
                1.0 - q[1, 1] - q[2, 2],
                0.0,
            ],
            [0.0, 0.0, 0.0, 1.0],
        ]
    )


# based on the 3D registration from https://github.com/cgohlke/transformations
def affine_matrix_from_points(v0, v1, shear=False, scale=True, usesvd=True):
    '''Return affine transform matrix to register two point sets.
    v0 and v1 are shape (ndims, -1) arrays of at least ndims non-homogeneous
    coordinates, where ndims is the dimensionality of the coordinate space.
    If shear is False, a similarity transformation matrix is returned.
    If also scale is False, a rigid/Euclidean traffansformation matrix
    is returned.
    By default the algorithm by Hartley and Zissermann [15] is used.
    If usesvd is True, similarity and Euclidean transformation matrices
    are calculated by minimizing the weighted sum of squared deviations
    (RMSD) according to the algorithm by Kabsch [8].
    Otherwise, and if ndims is 3, the quaternion based algorithm by Horn [9]
    is used, which is slower when using this Python implementation.
    The returned matrix performs rotation, translation and uniform scaling
    (if specified).'''
    
    v0 = np.array(v0, dtype=np.float64, copy=True)
    v1 = np.array(v1, dtype=np.float64, copy=True)

    ndims = v0.shape[0]
    if ndims < 2 or v0.shape[1] < ndims or v0.shape != v1.shape:
        raise ValueError("input arrays are of wrong shape or type")

    # move centroids to origin
    t0 = -np.mean(v0, axis=1)
    M0 = np.identity(ndims + 1)
    M0[:ndims, ndims] = t0
    v0 += t0.reshape(ndims, 1)
    t1 = -np.mean(v1, axis=1)
    M1 = np.identity(ndims + 1)
    M1[:ndims, ndims] = t1
    v1 += t1.reshape(ndims, 1)

    if shear:
        # Affine transformation
        A = np.concatenate((v0, v1), axis=0)
        u, s, vh = np.linalg.svd(A.T)
        vh = vh[:ndims].T
        B = vh[:ndims]
        C = vh[ndims: 2 * ndims]
        t = np.dot(C, np.linalg.pinv(B))
        t = np.concatenate((t, np.zeros((ndims, 1))), axis=1)
        M = np.vstack((t, ((0.0,) * ndims) + (1.0,)))
    elif usesvd or ndims != 3:
        # Rigid transformation via SVD of covariance matrix
        u, s, vh = np.linalg.svd(np.dot(v1, v0.T))
        # rotation matrix from SVD orthonormal bases
        R = np.dot(u, vh)
        if np.linalg.det(R) < 0.0:
            # R does not constitute right handed system
            R -= np.outer(u[:, ndims - 1], vh[ndims - 1, :] * 2.0)
            s[-1] *= -1.0
        # homogeneous transformation matrix
        M = np.identity(ndims + 1)
        M[:ndims, :ndims] = R
    else:
        # Rigid transformation matrix via quaternion
        # compute symmetric matrix N
        xx, yy, zz = np.sum(v0 * v1, axis=1)
        xy, yz, zx = np.sum(v0 * np.roll(v1, -1, axis=0), axis=1)
        xz, yx, zy = np.sum(v0 * np.roll(v1, -2, axis=0), axis=1)
        N = [
            [xx + yy + zz, 0.0, 0.0, 0.0],
            [yz - zy, xx - yy - zz, 0.0, 0.0],
            [zx - xz, xy + yx, yy - xx - zz, 0.0],
            [xy - yx, zx + xz, yz + zy, zz - xx - yy],
        ]
        # quaternion: eigenvector corresponding to most positive eigenvalue
        w, V = np.linalg.eigh(N)
        q = V[:, np.argmax(w)]
        # print (vector_norm(q), np.linalg.norm(q))
        q /= vector_norm(q)  # unit quaternion
        # homogeneous transformation matrix
        M = quaternion_matrix(q)

    if scale and not shear:
        # Affine transformation; scale is ratio of RMS deviations from centroid
        v0 *= v0
        v1 *= v1
        M[:ndims, :ndims] *= math.sqrt(np.sum(v1) / np.sum(v0))

    # move centroids back
    M = np.dot(np.linalg.inv(M1), np.dot(M, M0))
    M /= M[ndims, ndims]

    # print("transformation matrix Python Script: ", M)

    return M


# This is the IMC 3D error metric code
def register_by_Horn(ev_coord, gt_coord, ransac_threshold, inl_cf, strict_cf):
    '''Return the best similarity transforms T that registers 3D points pt_ev in <ev_coord> to
    the corresponding ones pt_gt in <gt_coord> according to a RANSAC-like approach for each
    threshold value th in <ransac_threshold>.
    
    Given th, each triplet of 3D correspondences is examined if not already present as strict inlier,
    a correspondence is a strict inlier if <strict_cf> * err_best < th, where err_best is the registration
    error for the best model so far.
    The minimal model given by the triplet is then refined using also its inliers if their total is greater
    than <inl_cf> * ninl_best, where ninl_best is th number of inliers for the best model so far. Inliers
    are 3D correspondences (pt_ev, pt_gt) for which the Euclidean distance |pt_gt-T*pt_ev| is less than th.'''
    
    # remove invalid cameras, the index is returned
    idx_cams = np.all(np.isfinite(ev_coord), axis=0)
    ev_coord = ev_coord[:, idx_cams]
    gt_coord = gt_coord[:, idx_cams]

    # initialization
    n = ev_coord.shape[1]
    r = ransac_threshold.shape[0]
    ransac_threshold = np.expand_dims(ransac_threshold, axis=0)
    ransac_threshold2 = ransac_threshold**2
    ev_coord_1 = np.vstack((ev_coord, np.ones(n)))

    max_no_inl = np.zeros((1, r))
    best_inl_err = np.full(r, np.Inf)
    best_transf_matrix = np.zeros((r, 4, 4))
    best_err = np.full((n, r), np.Inf)
    strict_inl = np.full((n, r), False)
    triplets_used = np.zeros((3, r))

    # run on camera triplets
    for ii in range(n-2):
        for jj in range(ii+1, n-1):
            for kk in range(jj+1, n):
                i = [ii, jj, kk]
                triplets_used_now = np.full((n), False)
                triplets_used_now[i] = True
                # if both ii, jj, kk are strict inliers for the best current model just skip
                if np.all(strict_inl[i]):
                    continue
                # get transformation T by Horn on the triplet camera center correspondences
                transf_matrix = affine_matrix_from_points(ev_coord[:, i], gt_coord[:, i], usesvd=False)
                # apply transformation T to test camera centres
                rotranslated = np.matmul(transf_matrix[:3], ev_coord_1)
                # compute error and inliers
                err = np.sum((rotranslated - gt_coord)**2, axis=0)
                inl = np.expand_dims(err, axis=1) < ransac_threshold2
                no_inl = np.sum(inl, axis=0)
                # if the number of inliers is close to that of the best model so far, go for refinement
                to_ref = np.squeeze(((no_inl > 2) & (no_inl > max_no_inl * inl_cf)), axis=0)
                for q in np.argwhere(to_ref):                        
                    qq = q[0]
                    if np.any(np.all((np.expand_dims(inl[:, qq], axis=1) == inl[:, :qq]), axis=0)):
                        # already done for this set of inliers
                        continue
                    # get transformation T by Horn on the inlier camera center correspondences
                    transf_matrix = affine_matrix_from_points(ev_coord[:, inl[:, qq]], gt_coord[:, inl[:, qq]])
                    # apply transformation T to test camera centres
                    rotranslated = np.matmul(transf_matrix[:3], ev_coord_1)
                    # compute error and inliers
                    err_ref = np.sum((rotranslated - gt_coord)**2, axis=0)
                    err_ref_sum = np.sum(err_ref, axis=0)
                    err_ref = np.expand_dims(err_ref, axis=1)
                    inl_ref = err_ref < ransac_threshold2
                    no_inl_ref = np.sum(inl_ref, axis=0)
                    # update the model if better for each threshold
                    to_update = np.squeeze((no_inl_ref > max_no_inl) | ((no_inl_ref == max_no_inl) & (err_ref_sum < best_inl_err)), axis=0)
                    if np.any(to_update):
                        triplets_used[0, to_update] = ii
                        triplets_used[1, to_update] = jj
                        triplets_used[2, to_update] = kk
                        max_no_inl[:, to_update] = no_inl_ref[to_update]
                        best_err[:, to_update] = np.sqrt(err_ref)
                        best_inl_err[to_update] = err_ref_sum
                        strict_inl[:, to_update] = (best_err[:, to_update] < strict_cf * ransac_threshold[:, to_update])
                        best_transf_matrix[to_update] = transf_matrix

    for i in range(r):
       print(f'Registered cameras {int(max_no_inl[0, i])}/{n} for threshold {ransac_threshold[0, i]}')

    best_model = {
        "valid_cams": idx_cams,        
        "no_inl": max_no_inl,
        "err": best_err,
        "triplets_used": triplets_used,
        "transf_matrix": best_transf_matrix}
    return best_model


# mAA computation
def mAA_on_cameras(err, thresholds, n, skip_top_thresholds, to_dec=3):
    '''mAA is the mean of mAA_i, where for each threshold th_i in <thresholds>, excluding the first <skip_top_thresholds values>,
    mAA_i = max(0, sum(err_i < th_i) - <to_dec>) / (n - <to_dec>)
    where <n> is the number of ground-truth cameras and err_i is the camera registration error for the best 
    registration corresponding to threshold th_i'''
    
    aux = err[:, skip_top_thresholds:] < np.expand_dims(np.asarray(thresholds[skip_top_thresholds:]), axis=0)
    return np.sum(np.maximum(np.sum(aux, axis=0) - to_dec, 0)) / (len(thresholds[skip_top_thresholds:]) * (n - to_dec))


# import data - no error handling in case float(x) fails
def get_camera_centers_from_df(df):
    out = {}
    for row in df.iterrows():
        row = row[1]
        fname = row['image_path']
        R = np.array([float(x) for x in (row['rotation_matrix'].split(';'))]).reshape(3,3)
        t = np.array([float(x) for x in (row['translation_vector'].split(';'))]).reshape(3)
        center = -R.T @ t
        out[fname] = center
    return out


def evaluate_rec(gt_df, user_df, inl_cf = 0.8, strict_cf=0.5, skip_top_thresholds=2, to_dec=3,
                 thresholds=[0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2]):
    ''' Register the <user_df> camera centers to the ground-truth <gt_df> camera centers and
    return the corresponding mAA as the average percentage of registered camera threshold.
    
    For each threshold value in <thresholds>, the best similarity transformation found which
    maximizes the number of registered cameras is employed. A camera is marked as registered
    if after the transformation its Euclidean distance to the corresponding ground-truth camera
    center is less than the mentioned threshold. Current measurements are in meter.
    
    Registration parameters:
    <inl_cf> coefficient to activate registration refinement, set to 1 to refine a new model
    only when it gives more inliers, to 0 to refine a new model always; high values increase
    speed but decrease precision.
    <strict_cf> threshold coefficient to define strict inliers for the best registration so far,
    new minimal models made up of strict inliers are skipped. It can vary from 0 (slower) to
    1 (faster); set to -1 to check exhaustively all the minimal model triplets.

    mAA parameters:
    <skip_top_thresholds> excluded lower thresholds in the mAA computation; in case of using
    heuristics for the registration, i.e. inl_cf!=0 and strict_cf!=-1, best model for lower
    threshold can be not the optimal, so skip them in the mAA computation.
    <to_dec> excludes the minimal model cameras from the computation of the mAA. Given the
    minimal model, i.e. three pairs of 3D correspondences, there is a high chance to register by
    a similarity transformation at any threshold, so do not account for mAA'''
    
    # get camera centers
    ucameras = get_camera_centers_from_df(user_df)
    gcameras = get_camera_centers_from_df(gt_df)    

    # the denominator for mAA ratio
    m = gt_df.shape[0]
    
    # get the image list to use
    good_cams = []
    for image_path in gcameras.keys():
        if image_path in ucameras.keys():
            good_cams.append(image_path)
        
    # put corresponding camera centers into matrices
    n = len(good_cams)
    u_cameras = np.zeros((3, n))
    g_cameras = np.zeros((3, n))
    
    ii = 0
    for i in good_cams:
        u_cameras[:, ii] = ucameras[i]
        g_cameras[:, ii] = gcameras[i]
        ii += 1
        
    # Horn camera centers registration, a different best model for each camera threshold
    model = register_by_Horn(u_cameras, g_cameras, np.asarray(thresholds), inl_cf, strict_cf)
    
    # transformation matrix
    print("\nTransformation matrix for maximum threshold")
    T = np.squeeze(model['transf_matrix'][-1])
    print(T)
    
    # mAA
    mAA = mAA_on_cameras(model["err"], thresholds, m, skip_top_thresholds, to_dec)
    # print(f'mAA = {mAA * 100 : .2f}% considering {m} input cameras - {to_dec}')
    return mAA


def score(solution: pd.DataFrame, submission: pd.DataFrame) -> float:
    '''The metric is an mean average accuracy between solution and submission camera centers.
    Prior to calculate the metric, a function performs exhaustive registration (like RANSAC, but
    not random, considering all possible configurations) to align the user camera system to the GT'''
    
    scenes = list(set(solution['dataset'].tolist()))
    results_per_dataset = []
    for dataset in scenes:
        print(f"\n*** {dataset} ***")
        start = time()
        gt_ds = solution[solution['dataset'] == dataset]
        user_ds = submission[submission['dataset'] == dataset]
        gt_ds = gt_ds.sort_values(by=['image_path'], ascending = True)
        user_ds = user_ds.sort_values(by=['image_path'], ascending = True)
        result = evaluate_rec(gt_ds, user_ds, inl_cf=0, strict_cf=-1, skip_top_thresholds=0, to_dec=3,
                 thresholds=translation_thresholds_meters_dict[dataset])
        end = time()
        print(f"\nmAA: {result*100}%")
        print("Running time: %s" % (end - start))        
        results_per_dataset.append(result)
    return float(np.array(results_per_dataset).mean())

# Run evaluation and log outputs 

In [88]:
gt_csv = 'ch_gt.csv'
user_csv = 'sub_sorted.csv'


gt_df = pd.read_csv(gt_csv)
user_df = pd.read_csv(user_csv)

gt_df = gt_df.rename(columns={'image_name': 'image_path'}) 
user_df = user_df.rename(columns={'image_name': 'image_path'}) 
gt_df['image_path'] = 'test/church/images/' + gt_df['image_path']


start = time()
res = score(gt_df, user_df)
end = time()

print(f"\nGlobal mAA: {res*100}%")
print("Total running time: %s" % (end - start))

# mAA: 7.017543859649122%


*** church ***


/tmp/ipykernel_76/4284531621.py:153: RuntimeWarning: divide by zero encountered in scalar divide
  M[:ndims, :ndims] *= math.sqrt(np.sum(v1) / np.sum(v0))
/tmp/ipykernel_76/4284531621.py:153: RuntimeWarning: invalid value encountered in multiply
  M[:ndims, :ndims] *= math.sqrt(np.sum(v1) / np.sum(v0))


Registered cameras 3/41 for threshold 0.025
Registered cameras 3/41 for threshold 0.05
Registered cameras 4/41 for threshold 0.1
Registered cameras 5/41 for threshold 0.2
Registered cameras 8/41 for threshold 0.5
Registered cameras 11/41 for threshold 1.0

Transformation matrix for maximum threshold
[[ -3.21449908   3.01231216   7.13788849 -25.69715   ]
 [ -3.17565659   6.53656363  -4.18867784  17.49865734]
 [ -7.06673063  -4.30764347  -1.36455372  36.2491671 ]
 [  0.           0.           0.           1.        ]]

mAA: 7.017543859649122%
Running time: 2.454275131225586

Global mAA: 7.017543859649122%
Total running time: 2.4546058177948
